In [1]:
import pandas as pd
from scipy.optimize import linprog
import numpy as np

In [2]:
df=pd.read_excel('data1.xlsx')
df

,Unnamed: 0,Имя,Цена,Сухое,Натура,Сырой,Крахмал,NDF,ADF,Ca,Phos
0,Имся,единица,руб,кг,кг,кг,кг,кг,кг,кг,кг
1,Ячмень зерно,кг,10.000,0.900,1.111,0.150,0.500,0.200,0.100,0.002,0.003
2,Кукуруза зерно,кг,15.000,0.800,1.250,0.100,0.600,0.250,0.150,0.003,0.004
3,Силос,кг,5.000,0.400,2.500,0.050,0.300,0.400,0.200,0.004,0.005
4,Соя шрот,кг,20.000,0.900,1.111,0.400,0.200,0.300,0.250,0.005,0.006


In [3]:
class method1():
    def __init__(self, nerva_lp, nerva_rp, costs, bnd, ratio=None):
        self.nerva_lp=nerva_lp
        self.nerva_rp=nerva_rp
        self.costs=costs
        self.bnd=bnd
        self.ratio=ratio
#         self.eq_lp=eq_lp
#         self.eq_rp=eq_rp
    
    def find_opt(self):
        if not self.ratio is None:
            A_ub=np.concatenate([self.nerva_lp, self.ratio], axis=0)
            b_ub=np.concatenate([self.nerva_rp, np.zeros(self.ratio.shape[0])], axis=0)
        else:
            A_ub=self.nerva_lp
            b_ub=self.nerva_rp
        opt = linprog(c=self.costs, 
        A_ub=A_ub,
        b_ub=b_ub,
        #A_eq=eq_lp, b_eq=eq_rp, 
        bounds=self.bnd,
        method="highs")
        return {'success':opt.success, 'x':opt.x, 'fun':opt.fun}
    
    def bnd2condition(comps_bnd):
        cond_l=[]
        cond_r=[]
        for num, i in enumerate(comps_bnd):
            if i[0]:
                cond_l.append(-main_mat[:, num])
                cond_r.append(-i[0])
            if i[1]:
                cond_l.append(main_mat[:, num])
                cond_r.append(i[1])
        cond_l=np.array(cond_l)
        cond_r=np.array(cond_r)
        return cond_l, cond_r
    
    def get_data(products, comps, sootnosheniya):
        main_mat=matrix[products, :][:, comps]
        for_ratio=np.zeros([len(sootnosheniya), len(products)])
        for num, i in enumerate(sootnosheniya):
            chisl=0
            znam=0
            idx1=0
            idx2=0
            for num1, j in enumerate(i):
                if j!=0:
                    if j!=1 and j!=-1:
                        chisl=j
                        idx1=num1
                    else:
                        znam=j
                        idx2=num1
            for_ratio[num]=main_mat[:, idx2]*chisl-main_mat[:, idx1]*znam
        return main_mat, for_ratio
    
    def change_bnd(self):
        #self.bnd=np.stack([np.where(self.bnd[:, 0]-[1/costs[0]]<0, 0, self.bnd[:, 0]-[1/costs[1]]), self.bnd[:, 1]+[1/costs[0]]], axis=1)
        self.nerva_rp=np.where(test1.nerva_rp[:]>0, test1.nerva_rp[:]*1.1, test1.nerva_rp[:]/1.1)
        #print(self.nerva_rp)
        
    def find_rad(self, num):
        counter=0
        if num==1:
            #Постепенно расширяет границы условий
            while not self.find_opt()['success']:
                self.change_bnd()
                print('№', counter, self.nerva_rp)
                counter+=1
        elif num==2:
#             A_ub=self.nerva_lp
#             b_ub=np.where(test1.nerva_rp[:]>0, float('inf'), 0)
            opt = linprog(c=self.costs, 
            A_ub=[[0]*len(self.nerva_lp)],
            b_ub=[0],
            #A_eq=eq_lp, b_eq=eq_rp, 
            bounds=self.bnd,
            method="highs")
            return {'success':opt.success, 'x':opt.x, 'fun':opt.fun}
            
        return self.find_opt()

In [4]:
datas=df.drop([0], axis=0).drop(['Unnamed: 0', 'Имя'], axis=1).astype(float).to_numpy()
datas

array([[1.000e+01, 9.000e-01, 1.111e+00, 1.500e-01, 5.000e-01, 2.000e-01,
        1.000e-01, 2.000e-03, 3.000e-03],
       [1.500e+01, 8.000e-01, 1.250e+00, 1.000e-01, 6.000e-01, 2.500e-01,
        1.500e-01, 3.000e-03, 4.000e-03],
       [5.000e+00, 4.000e-01, 2.500e+00, 5.000e-02, 3.000e-01, 4.000e-01,
        2.000e-01, 4.000e-03, 5.000e-03],
       [2.000e+01, 9.000e-01, 1.111e+00, 4.000e-01, 2.000e-01, 3.000e-01,
        2.500e-01, 5.000e-03, 6.000e-03]])

In [5]:
costs=datas[:, 0]
costs

array([10., 15.,  5., 20.])

In [6]:
voda=datas[:, 2:3]
voda

array([[1.111],
       [1.25 ],
       [2.5  ],
       [1.111]])

In [7]:
componets_for_div=datas[:, 3:]
componets_for_div

array([[0.15 , 0.5  , 0.2  , 0.1  , 0.002, 0.003],
       [0.1  , 0.6  , 0.25 , 0.15 , 0.003, 0.004],
       [0.05 , 0.3  , 0.4  , 0.2  , 0.004, 0.005],
       [0.4  , 0.2  , 0.3  , 0.25 , 0.005, 0.006]])

In [8]:
divided=componets_for_div/voda
divided

array([[0.1350135 , 0.450045  , 0.180018  , 0.090009  , 0.00180018,
        0.00270027],
       [0.08      , 0.48      , 0.2       , 0.12      , 0.0024    ,
        0.0032    ],
       [0.02      , 0.12      , 0.16      , 0.08      , 0.0016    ,
        0.002     ],
       [0.360036  , 0.180018  , 0.270027  , 0.2250225 , 0.00450045,
        0.00540054]])

In [9]:
divided=np.transpose(np.transpose(np.concatenate([datas[:, 2:3], divided], axis=1)))
divided

array([[1.11100000e+00, 1.35013501e-01, 4.50045005e-01, 1.80018002e-01,
        9.00090009e-02, 1.80018002e-03, 2.70027003e-03],
       [1.25000000e+00, 8.00000000e-02, 4.80000000e-01, 2.00000000e-01,
        1.20000000e-01, 2.40000000e-03, 3.20000000e-03],
       [2.50000000e+00, 2.00000000e-02, 1.20000000e-01, 1.60000000e-01,
        8.00000000e-02, 1.60000000e-03, 2.00000000e-03],
       [1.11100000e+00, 3.60036004e-01, 1.80018002e-01, 2.70027003e-01,
        2.25022502e-01, 4.50045005e-03, 5.40054005e-03]])

In [10]:
matrix=np.concatenate([datas[:,1].reshape((4, 1)), divided], axis=1)
matrix

array([[9.00000000e-01, 1.11100000e+00, 1.35013501e-01, 4.50045005e-01,
        1.80018002e-01, 9.00090009e-02, 1.80018002e-03, 2.70027003e-03],
       [8.00000000e-01, 1.25000000e+00, 8.00000000e-02, 4.80000000e-01,
        2.00000000e-01, 1.20000000e-01, 2.40000000e-03, 3.20000000e-03],
       [4.00000000e-01, 2.50000000e+00, 2.00000000e-02, 1.20000000e-01,
        1.60000000e-01, 8.00000000e-02, 1.60000000e-03, 2.00000000e-03],
       [9.00000000e-01, 1.11100000e+00, 3.60036004e-01, 1.80018002e-01,
        2.70027003e-01, 2.25022502e-01, 4.50045005e-03, 5.40054005e-03]])

Первый индекс описывает продукт
1	Ячмень зерно	
2	Кукуруза зерно	
3	Силос	
4	Соя шрот
Второй индекс описывает компонент (кроме натурального, ушёл как коэф.)
1 Сухое
2 Натур
3 Сырой
4 Крахмал
5 NDF
6 ADF
7 Ca
8 Phos

## Пример1

In [11]:
products=[0, 1, 2]#Номера продуктов
bnd=[[0, 5], [0, 10], [0, float('inf')]]#Ограничения на продукты
comps=[0, 1, 2, 3]#Номера компонентов
comps_bnd=[[10, None],[20, None],[1.5, None], [6, None]]#Ограничения на компоненты
sootnosheniya=[[0.6, 1, 0, 0], [1, 0, 0.12, 0]]#соотношения компонентов
#комп1/комп2>=0.6 комп3/комп1>=0.12

Если условие >=, то данные идут с плюсом.
Если <=, то надо умножить c -1.

In [12]:
main_mat, for_ratio=method1.get_data(products, comps, sootnosheniya)
cond_l, cond_r=method1.bnd2condition(comps_bnd)

In [13]:
test1=method1(cond_l, cond_r, costs[products], bnd=bnd, ratio=for_ratio)
opt=test1.find_opt()
opt

{'success': False, 'x': None, 'fun': None}

In [14]:
test1=method1(cond_l, cond_r, costs[products], bnd=bnd)
opt=test1.find_opt()
opt

{'success': True,
 'x': array([ 5.        , 10.        ,  1.24662466]),
 'fun': 206.23312331233123}

In [15]:
#компоненты
np.matmul(opt['x'], main_mat)

array([12.99864986, 21.17156166,  1.5       ,  7.19981998])

In [16]:
#Проверка соотношения
proport=np.matmul(opt['x'], main_mat)
print(proport[0]/proport[1], proport[0]/proport[1]>=0.6)
print(proport[2]/proport[0], proport[2]/proport[0]>=0.12)

0.613967456727549 True
0.11539660007616939 False


## Пример 2

In [17]:
products=[0, 1, 2]#Номера продуктов
bnd=[[0, 5], [0, 10], [0, float('inf')]]#Ограничения на продукты
comps=[0, 1, 2, 3]#Номера компонентов
comps_bnd=[[10, None],[None, None],[2, None], [None, None]]#Ограничения на компоненты
sootnosheniya=[[0.6, 1, 0, 0], [1, 0, 0.0, 0]]#соотношения компонентов
#комп1/комп2>=0.6 комп3/комп1>=0.12
main_mat, for_ratio=method1.get_data(products, comps, sootnosheniya)
cond_l, cond_r=method1.bnd2condition(comps_bnd)

In [18]:
test1=method1(cond_l, cond_r, costs[products], bnd=bnd, ratio=for_ratio)
#test1=method1(cond_l, cond_r, costs[products], bnd=bnd)
opt=test1.find_opt()
opt

{'success': False, 'x': None, 'fun': None}

In [19]:
test1=method1(cond_l, cond_r, costs[products], bnd=bnd)
opt=test1.find_opt()
opt

{'success': True,
 'x': array([ 5.        , 10.        , 26.24662466]),
 'fun': 331.2331233123312}

In [20]:
#компоненты
np.matmul(opt['x'], main_mat)

array([22.99864986, 83.67156166,  2.        , 10.19981998])

In [21]:
#Проверка соотношения
proport=np.matmul(opt['x'], main_mat)
print(proport)

[22.99864986 83.67156166  2.         10.19981998]


## Пример 3

In [22]:
products=[0, 1, 2]#Номера продуктов
bnd=[[5, 5], [8.75, 8.75], [1.25, 1.25]]#Ограничения на продукты
comps=[0, 1, 2, 3]#Номера компонентов
comps_bnd=[[10, None],[15, None],[1.3, None], [None, None]]#Ограничения на компоненты
sootnosheniya=[[0.8, 1, 0, 0],  [0.2, 0, 0.0, 1]]#соотношения компонентов
#комп1/комп2>=0.6 комп3/комп1>=0.12
main_mat, for_ratio=method1.get_data(products, comps, sootnosheniya)
cond_l, cond_r=method1.bnd2condition(comps_bnd)

In [23]:
test1=method1(cond_l, cond_r, costs[products], bnd=bnd, ratio=for_ratio)
opt=test1.find_opt()
opt

{'success': False, 'x': None, 'fun': None}

In [24]:
test1=method1(cond_l, cond_r, costs[products], bnd=bnd)
opt=test1.find_opt()
opt

{'success': True, 'x': array([5.  , 8.75, 1.25]), 'fun': 187.5}

In [25]:
#компоненты
np.matmul(opt['x'], main_mat)

array([12.        , 19.6175    ,  1.40006751,  6.60022502])

In [26]:
#Проверка соотношения
proport=np.matmul(opt['x'], main_mat)
print(proport[0]/proport[1], proport[3]/proport[0])

0.6116987383713521 0.5500187518751876
